In [14]:
import numpy as np
import math

KNN

In [2]:
def counter(x):
    one_count = 0
    zero_count = 0
    for i in x:
        if i==0:
            zero_count+=1
        elif i==1:
            one_count+=1
        else:
            return 'Invalid type :'+i
    if zero_count >one_count:
        return 0 
    elif one_count>zero_count:
        return 1
    
class Knn:
    def __init__(self,n_neighbors,metrics = 'Euclidean'):
        #class initilization
        self.n_neighbors = n_neighbors
        if(metrics=='Euclidean'):
            self.metrics =self.euclidean
        elif(metrics=='Manhattan'):
            self.metrics = self.manhattan
        elif(metrics=='Minkowski'):
            self.metrics = self.minkowski
    def euclidean(self,x1,x2):
        #to calculate Euclidean distance
        if(x1.shape==x2.shape):
            distance = sum((px - qx) ** 2.0 for px, qx in zip(x1, x2))
            return math.sqrt(distance)
        else:
            return -1
    def manhattan(self,x1,x2):
        if(x1.shape==x2.shape):
            distance = sum((px - qx) for px, qx in zip(x1, x2))
            return abs(distance)
        else:
            return -1
    def minkowski(self,x1,x2):
        if(x1.shape==x2.shape):
            distance = sum((px - qx) ** 3.0 for px, qx in zip(x1, x2))
            return np.cbrt(distance)
        else:
            return -1
    def fit(self,x_train,y_train):
        self.x_train=x_train
        self.y_train=y_train
    def predict(self,x_test):
        y_predict = []
        y_train = self.y_train
        for x in x_test:
            distance = []
            for x_train in self.x_train:
                dist = self.metrics(x,x_train)
                distance.append(dist)
            distance = np.array(distance).reshape(len(self.x_train),1)
            np_dists = np.hstack([distance,y_train])
            # Sort distances, and n closest points
            np_dists = np_dists[np_dists[:,0].argsort()][:self.n_neighbors]
            # Create counter object to track the labels of k closest neighbors
            # Get Most common label of all the nearest neighbors
            prediction = counter(np_dists[:,1])
            y_predict.append(prediction)
        return y_predict

Naive Bayes

In [3]:
class GaussianNaiveBayes:

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)

        self.means = np.zeros((n_classes, n_features), dtype=np.float64)
        self.var = np.zeros((n_classes, n_features), dtype=np.float64)
        self.priors =  np.zeros(n_classes, dtype=np.float64)

        # calculating the mean, variance and prior P(H) for each class
        for i, c in enumerate(self.classes):
            X_class=[]
            for j in range(len(X)):
                if y[j]==c:
                    X_class.append(X[j])
            X_class = np.array(X_class)
            self.means[i, :] = X_class.mean(axis=0)
            self.var[i, :] = X_class.var(axis=0)
            self.priors[i] = X_class.shape[0] / float(n_samples)

    # calculating the likelihood, P(E|H)
    def likelihood(self, class_idx, x):
        mean = self.means[class_idx]
        var = self.var[class_idx]
        num = np.exp(- (x-mean)**2 / (2 * var))
        denom = np.sqrt(2 * np.pi * var)
        return num / denom

    def predict(self, X):
        y_pred = [self.classify(x) for x in X]
        return np.array(y_pred)

    def classify(self, x):
        posteriors = []

        # calculating posterior probability for each class
        for i, c in enumerate(self.classes):
            posteriors.append(np.log(self.priors[i])+np.sum(np.log(self.likelihood(i, x))))
            
        # return the class with highest posterior probability
        return self.classes[np.argmax(posteriors)]

In [4]:
def clean_data(row):
    return row.replace('(', '').replace('W','1').replace('M','0').replace(')', '').replace(' ', '').strip().split(',')
def readFile(path):
    with open(path, 'r') as file:
        data = file.readlines()
        clean = list(map(clean_data, data))
        file.close()
    input_np = np.array(clean)
    return input_np

training = './dataset/1a-training.txt'
test = './dataset/1a-test.txt'

train_np = readFile(training).astype(float)
test_np = readFile(test).astype(float)
x_train =train_np[: , 0:3]
y_train = train_np[:, 3].reshape(len(x_train),1)
x_test = test_np

KNN implementation on Traning and testing data with k=1,3,7

In [5]:
k1 = Knn(1,'Euclidean')
nb = GaussianNaiveBayes()
nb.fit(x_train,y_train)
k1.fit(x_train,y_train)
print('KNN for k=1 :',k1.predict(x_test))
print('Naive Bayes predction :', nb.predict(x_test))

KNN for k=1 : [1, 1, 1, 1]
Naive Bayes predction : [1. 1. 1. 1.]


In [6]:
k3 = Knn(3,'Manhattan')
k3.fit(x_train,y_train)
print('KNN for k=3 :',k3.predict(x_test))

KNN for k=3 : [1, 0, 1, 1]


In [7]:
k7 = Knn(7,'Euclidean')
k7.fit(x_train,y_train)
print('KNN for k=7 :',k7.predict(x_test))

KNN for k=7 : [1, 0, 1, 1]


Leave one-out Evaluation

In [8]:
large_data = './dataset/1c-data.txt'
inpu=readFile(large_data).astype(float)

In [9]:
x_train =inpu[: , 0:3]
y_train = inpu[:, 3].reshape(len(x_train),1)
print(x_train.shape,y_train.shape)

(120, 3) (120, 1)


In [10]:
def leave_out(model,x_train,y_train):
    count =0
    for i in range(len(x_train)):
        x_test = np.array(x_train[i]).reshape(1,x_train.shape[1])
        y_test = np.array(y_train[i])
        x_train_i= x_train[0:i, :]
        y_train_i = y_train[0:i,:]
        x_train_j = x_train[i+1:,:]
        y_train_j = y_train[i+1:,:]
        x_train_input = np.vstack((x_train_i,x_train_j))
        y_train_input = np.vstack((y_train_i,y_train_j))

        model.fit(x_train_input,y_train_input)
        if(model.predict(x_test)==y_test):
            count+=1
    return count/len(x_train)*100

Leave_out evaluation for k=1, 3, 5, 7, 9, and 11

In [11]:
knd = Knn(1)
nav = GaussianNaiveBayes()
print('Accuracy score of KNN for k= 1 :',leave_out(knd,x_train,y_train))
print('Accuracy score for Naive Bayes :',leave_out(nav,x_train,y_train))
knd = Knn(3)
print('Accuracy score of KNN for k= 3 :',leave_out(knd,x_train,y_train))
knd = Knn(5)
print('Accuracy score of KNN for k= 5 :',leave_out(knd,x_train,y_train))
knd = Knn(7)
print('Accuracy score of KNN for k= 7 :',leave_out(knd,x_train,y_train))
knd = Knn(9)
print('Accuracy score of KNN for k= 9 :',leave_out(knd,x_train,y_train))
knd = Knn(11)
print('Accuracy score of KNN for k= 11 :',leave_out(knd,x_train,y_train))


Accuracy score of KNN for k= 1 : 55.00000000000001
Accuracy score for Naive Bayes : 70.0
Accuracy score of KNN for k= 3 : 61.66666666666667
Accuracy score of KNN for k= 5 : 61.66666666666667
Accuracy score of KNN for k= 7 : 60.83333333333333
Accuracy score of KNN for k= 9 : 63.33333333333333
Accuracy score of KNN for k= 11 : 59.166666666666664


In [12]:
x_train =inpu[: , 0:2]#age data removed
x_train.shape

(120, 2)

In [13]:
knd = Knn(1)
nav = GaussianNaiveBayes()
print('Accuracy score of KNN for k= 1 :',leave_out(knd,x_train,y_train))

print('Accuracy score for Naive Bayes :',leave_out(nav,x_train,y_train))
knd = Knn(3)
print('Accuracy score of KNN for k= 3 :',leave_out(knd,x_train,y_train))
knd = Knn(5)
print('Accuracy score of KNN for k= 5 :',leave_out(knd,x_train,y_train))
knd = Knn(7)
print('Accuracy score of KNN for k= 7 :',leave_out(knd,x_train,y_train))
knd = Knn(9)
print('Accuracy score of KNN for k= 9 :',leave_out(knd,x_train,y_train))
knd = Knn(11)
print('Accuracy score of KNN for k= 11 :',leave_out(knd,x_train,y_train))

Accuracy score of KNN for k= 1 : 62.5
Accuracy score for Naive Bayes : 70.83333333333334
Accuracy score of KNN for k= 3 : 70.83333333333334
Accuracy score of KNN for k= 5 : 65.0
Accuracy score of KNN for k= 7 : 63.33333333333333
Accuracy score of KNN for k= 9 : 60.0
Accuracy score of KNN for k= 11 : 57.49999999999999
